In [1]:
import torch
import json
from tqdm import tqdm
from itertools import product as p
import numpy as np
import pickle

# Load fake activations

In [2]:
class A:
    pass
self = A()

In [3]:
self.num_neurons_d = {} # {fname, int}
self.representations_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
representation_files = [f1, f2, f3]

In [4]:
# initialize `num_neurons_d`, `representations_d` with fake data
n1, n2, n3 = 100, 80, 70
nword = 1000
t1 = torch.randn(nword, n1)
t2 = torch.randn(nword, n2)
t3 = torch.randn(nword, n3)
self.num_neurons_d[f1] = n1
self.num_neurons_d[f2] = n2
self.num_neurons_d[f3] = n3
self.representations_d[f1] = t1
self.representations_d[f2] = t2
self.representations_d[f3] = t3

In [5]:
self.device = torch.device('cpu')
self.op = min

In [6]:
def compute_correlations(self):
    # Set `means_d`, `stdevs_d`
    means_d = {}
    stdevs_d = {}
    for network in tqdm(self.representations_d, desc='mu, sigma'):
        t = self.representations_d[network]

        means_d[network] = t.mean(0, keepdim=True)
        stdevs_d[network] = (t - means_d[network]).pow(2).mean(0, keepdim=True).pow(0.5)

    # Set `self.corrs` : {network: {other: [corr]}}
    # Set `self.pairs` : {network: {other: [pair]}}
    # pair is index of neuron in other network
    # Set `self.similarities` : {network: {other: sim}}
    self.corrs = {network: {} for network in
                         self.representations_d}
    self.pairs = {network: {} for network in
                         self.representations_d}
    self.similarities = {network: {} for network in
                     self.representations_d}
    num_words = next(iter(self.representations_d.values())).size()[0]
    for network, other_network in tqdm(p(self.representations_d,
                                         self.representations_d),
                                         desc='correlate',
                                         total=len(self.representations_d)**2):
        if network == other_network:
            continue

        if other_network in self.corrs[network]: 
            continue

        device = self.device

        t1 = self.representations_d[network].to(device) # "tensor"
        t2 = self.representations_d[other_network].to(device)
        m1 = means_d[network].to(device) # "means"
        m2 = means_d[other_network].to(device)
        s1 = stdevs_d[network].to(device) # "stdevs"
        s2 = stdevs_d[other_network].to(device)

        covariance = (torch.mm(t1.t(), t2) / num_words # E[ab]
                      - torch.mm(m1.t(), m2)) # E[a]E[b]
        correlation = covariance / torch.mm(s1.t(), s2)
        correlation = correlation.cpu().numpy()
        correlation = np.abs(correlation)

        self.corrs[network][other_network] = correlation.max(axis=1)
        self.corrs[other_network][network] = correlation.max(axis=0)
        
        self.similarities[network][other_network] = self.corrs[network][other_network].mean()
        self.similarities[other_network][network] = self.corrs[other_network][network].mean()
        
        self.pairs[network][other_network] = correlation.argmax(axis=1)
        self.pairs[other_network][network] = correlation.argmax(axis=0)

    # Set `self.neuron_sort` : {network, sorted_list}
    # Set `self.neuron_notated_sort` : {network: [(neuron, {other: (corr, pair)})]}
    self.neuron_sort = {} 
    self.neuron_notated_sort = {}
    for network in tqdm(self.representations_d, desc='annotation'):
        self.neuron_sort[network] = sorted(
            range(self.num_neurons_d[network]), 
            key=lambda i: self.op(
                self.corrs[network][other][i] for other in self.corrs[network]
            ), 
            reverse=True,
        )
        self.neuron_notated_sort[network] = [
            (
                neuron,
                {
                    other : (
                        self.corrs[network][other][neuron], 
                        self.pairs[network][other][neuron],
                    ) 
                    for other in self.corrs[network]
                }
            ) 
            for neuron in self.neuron_sort[network]
        ]

In [7]:
compute_correlations(self)

annotation: 100%|████████████████████████████| 3/3 [00:00<00:00, 2230.62it/s]


# 

In [8]:
output_file = "temp"

In [13]:
output = {
    "corrs" : self.corrs, 
    "pairs" : self.pairs,
    "similarities" : self.similarities,
    "neuron_sort" : self.neuron_sort, 
    "neuron_notated_sort" : self.neuron_notated_sort,
}

In [14]:
with open(output_file, "wb") as f:
    pickle.dump(output, f)

In [15]:
with open(output_file, "rb") as f:
    d = pickle.load(f)

In [17]:
d['similarities']

{'foo': {'bar': 0.084357984, 'baz': 0.083409265},
 'bar': {'foo': 0.08770261, 'baz': 0.08129728},
 'baz': {'foo': 0.08705892, 'bar': 0.08281621}}

In [18]:
# full
def write_correlations(self, output_file):
    output = {
        "corrs" : self.corrs, 
        "pairs" : self.pairs,
        "similarities" : self.similarities,
        "neuron_sort" : self.neuron_sort, 
        "neuron_notated_sort" : self.neuron_notated_sort,
    }
    
    with open(output_file, "wb") as f:
        pickle.dump(output, f)

In [19]:
write_correlations(self, output_file)